In [113]:
# Se importan las librerias y módulos necesarios para crear el DF
import pandas as pd
import numpy as np
import re

In [114]:
# Importamos el archivo csv
sharks = pd.read_csv("input/attacks.csv", engine = 'python')

In [115]:
# Usamos shape para saber el número total de registros y columnas
sharks.shape

(25723, 24)

In [116]:
# Visualizamos las columnas para seleccionar posteriormente las que serán significativas para nuestra hipótesis
sharks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [117]:
# Visualizamos el DF para conocer los datos con los que vamos a trabajar
sharks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


## Hipótesis 1: 
“El mayor número de muertes por ataques de tiburón se produjo a surfistas"

In [118]:
# Sobreescribo el DF para quedarme con las columnas que son más significativas
df_resume = sharks[['Case Number', 'Year', 'Type', 'Country', 'Area', 'Activity', 'Injury', 'Species ', 'Fatal (Y/N)']]

In [119]:
df_resume.head()

,Case Number,Year,Type,Country,Area,Activity,Injury,Species,Fatal (Y/N)
0,2018.06.25,2018.0,Boating,USA,California,Paddling,"No injury to occupant, outrigger canoe and pad...",White shark,N
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,Standing,Minor injury to left thigh,NaN,N
2,2018.06.09,2018.0,Invalid,USA,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,NaN,N
3,2018.06.08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,Minor injury to lower leg,2 m shark,N
4,2018.06.04,2018.0,Provoked,MEXICO,Colima,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,"Tiger shark, 3m",N


In [120]:
# Quitamos el espacio que hay detrás de la columna 'Species'
df_resume.rename(columns=lambda x: x.rstrip(), inplace=True)
df_resume.columns

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:4290: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Index(['Case Number', 'Year', 'Type', 'Country', 'Area', 'Activity', 'Injury',
       'Species', 'Fatal (Y/N)'],
      dtype='object')

In [121]:
df_resume.dtypes

Case Number     object
Year           float64
Type            object
Country         object
Area            object
Activity        object
Injury          object
Species         object
Fatal (Y/N)     object
dtype: object

In [122]:
df_resume.shape

(25723, 9)

In [123]:
df_resume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Case Number  8702 non-null   object 
 1   Year         6300 non-null   float64
 2   Type         6298 non-null   object 
 3   Country      6252 non-null   object 
 4   Area         5847 non-null   object 
 5   Activity     5758 non-null   object 
 6   Injury       6274 non-null   object 
 7   Species      3464 non-null   object 
 8   Fatal (Y/N)  5763 non-null   object 
dtypes: float64(1), object(8)
memory usage: 1.8+ MB


In [124]:
# Comprobamos si existen duplicados para quedarnos con los datos correctos
df_resume['Case Number'].duplicated().value_counts()

True     19435
False     6288
Name: Case Number, dtype: int64

In [125]:
# Comprobamos si el número de duplicados afecta al resto de columnas
df_resume[df_resume['Case Number'].duplicated(keep=False)][['Case Number', 'Year', 'Country', 'Area', 'Injury', 'Species']]

,Case Number,Year,Country,Area,Injury,Species
522,2014.08.02,2014.0,USA,Florida,Foot bitten,NaN
523,2014.08.02,2014.0,USA,Florida,Laceration to ankle,NaN
615,2013.10.05,2013.0,USA,California,Laceration to thigh,"White shark, 8' to 10'"
616,2013.10.05,2013.0,USA,Florida,Lacerations to right forearm,Small bull shark
746,2012.09.02.b,2012.0,USA,Florida,Puncture wounds to calf and hand,3.5' to 4' shark
...,...,...,...,...,...,...
25717,NaN,NaN,NaN,NaN,NaN,NaN
25718,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
# Creamos una función para eliminar sólo aquellos casos en los que realmente hay un duplicado

index = []

for i in df_resume[df_resume['Case Number'].duplicated()].index:
    index.append(i)

df_resume = df_resume.drop(index,axis=0)

for i in index:
    df_resume[df_resume['Case Number'][i]] = df_resume[df_resume['Case Number'][i] +" z"]

KeyError: 523

In [127]:
df_resume.duplicated().value_counts()

False    6288
dtype: int64

In [128]:
df_resume

,Case Number,Year,Type,Country,Area,Activity,Injury,Species,Fatal (Y/N)
0,2018.06.25,2018.0,Boating,USA,California,Paddling,"No injury to occupant, outrigger canoe and pad...",White shark,N
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,Standing,Minor injury to left thigh,NaN,N
2,2018.06.09,2018.0,Invalid,USA,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,NaN,N
3,2018.06.08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,Minor injury to lower leg,2 m shark,N
4,2018.06.04,2018.0,Provoked,MEXICO,Colima,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,"Tiger shark, 3m",N
...,...,...,...,...,...,...,...,...,...
6299,ND.0003,0.0,Unprovoked,USA,North Carolina,Swimming,FATAL,NaN,Y
6300,ND.0002,0.0,Unprovoked,PANAMA,NaN,NaN,FATAL,NaN,Y
6301,ND.0001,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,Swimming,"FATAL. ""Shark bit him in half, carrying away t...",NaN,Y
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
# Sobreescribo el DF puesto que la columna Species no aporta consistencia debido al número de nulos
null_cols = df_resume.isnull().sum()
null_cols

Case Number       1
Year              4
Type              6
Country          52
Area            456
Activity        544
Injury           30
Species        2834
Fatal (Y/N)     537
dtype: int64

In [130]:
null_displ = df_resume[(df_resume['Species'].isnull()==True)]
df_resume[['Species']] = df_resume[['Species']].fillna(0)

In [131]:
#  Eliminamos las columnas que tienen más de 2000 valores nulos en ellas. Agregamos estos nombres de columna a una lista, y luego pasaremos esas columnas al método de colocación 
# e indicaremos que queremos que se eliminen las columnas (no las filas) estableciendo el parámetro del eje en 1
drop_cols = list(null_cols[null_cols > 2000].index)
df_resume = df_resume.drop(drop_cols, axis=1)

In [132]:
df_resume.head(20)

,Case Number,Year,Type,Country,Area,Activity,Injury,Fatal (Y/N)
0,2018.06.25,2018.0,Boating,USA,California,Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,Standing,Minor injury to left thigh,N
2,2018.06.09,2018.0,Invalid,USA,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,Minor injury to lower leg,N
4,2018.06.04,2018.0,Provoked,MEXICO,Colima,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N
5,2018.06.03.b,2018.0,Unprovoked,AUSTRALIA,New South Wales,Kite surfing,"No injury, board bitten",N
6,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,Swimming,FATAL,Y
7,2018.05.27,2018.0,Unprovoked,USA,Florida,Fishing,Minor injury to foot. PROVOKED INCIDENT,N
8,2018.05.26.b,2018.0,Unprovoked,USA,Florida,Walking,Lower left leg bitten,N
9,2018.05.26.a,2018.0,Unprovoked,USA,Florida,Standing,Minor injury to foot,N


In [140]:
df_resume['Year'] = df_resume['Year'].fillna(0).astype(int)

In [141]:
df_resume

,Case Number,Year,Type,Country,Area,Activity,Injury,Fatal (Y/N)
0,2018.06.25,2018,Boating,USA,California,Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,2018,Unprovoked,USA,Georgia,Standing,Minor injury to left thigh,N
2,2018.06.09,2018,Invalid,USA,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,2018,Unprovoked,AUSTRALIA,New South Wales,Surfing,Minor injury to lower leg,N
4,2018.06.04,2018,Provoked,MEXICO,Colima,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...
6299,ND.0003,0,Unprovoked,USA,North Carolina,Swimming,FATAL,Y
6300,ND.0002,0,Unprovoked,PANAMA,NaN,NaN,FATAL,Y
6301,ND.0001,0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,Swimming,"FATAL. ""Shark bit him in half, carrying away t...",Y
6302,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
# Comprobamos cuál es la actividad que tiene mayor impacto en los ataques de tiburón
df_resume.Activity.value_counts()

Surfing                                                                                                      968
Swimming                                                                                                     869
Fishing                                                                                                      430
Spearfishing                                                                                                 331
Bathing                                                                                                      162
                                                                                                            ... 
Removing shark from net                                                                                        1
Fell overboard & swimming                                                                                      1
Fishing, removing the shark from his line                                                       

In [143]:
# Filtramos por la Actividad Surfing para quedarnos con los datos que nos interesan
df_sharks = df_resume[df_resume.Activity == 'Surfing']
df_sharks

,Case Number,Year,Type,Country,Area,Activity,Injury,Fatal (Y/N)
2,2018.06.09,2018,Invalid,USA,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,2018,Unprovoked,AUSTRALIA,New South Wales,Surfing,Minor injury to lower leg,N
15,2018.05.12,2018,Unprovoked,SOUTH AFRICA,Eastern Cape Province,Surfing,"Minor injury, marks on board",N
16,2018.05.09,2018,Questionable,AUSTRALIA,New South Wales,Surfing,"No injury, surfboard damaged",N
19,2018.04.28.a,2018,Unprovoked,AUSTRALIA,Western Australia,Surfing,"No injury, knocked off board by shark",N
...,...,...,...,...,...,...,...,...
5235,1922.01.04,1922,Unprovoked,AUSTRALIA,New South Wales,Surfing,"FATAL, disappeared, then his shark-bitten rema...",Y
5280,1918.03.22,1918,Unprovoked,AUSTRALIA,New South Wales,Surfing,"Severe laceration to arm, necessitating surgic...",N
6097,1828.00.00,1828,Unprovoked,USA,Hawaii,Surfing,FATAL,Y
6143,1779.00.00,1779,Unprovoked,USA,Hawaii,Surfing,"FATAL, buttock lacerated",Y


In [145]:
df_sharks.head()

,Case Number,Year,Type,Country,Area,Activity,Injury,Fatal (Y/N)
2,2018.06.09,2018,Invalid,USA,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,2018,Unprovoked,AUSTRALIA,New South Wales,Surfing,Minor injury to lower leg,N
15,2018.05.12,2018,Unprovoked,SOUTH AFRICA,Eastern Cape Province,Surfing,"Minor injury, marks on board",N
16,2018.05.09,2018,Questionable,AUSTRALIA,New South Wales,Surfing,"No injury, surfboard damaged",N
19,2018.04.28.a,2018,Unprovoked,AUSTRALIA,Western Australia,Surfing,"No injury, knocked off board by shark",N


## Hipótesis 2
Estados Unidos es el país con mas ataques de tiburones a surfistas

In [169]:
df_sharks.Country.value_counts()

USA                                      563
AUSTRALIA                                195
SOUTH AFRICA                              95
BRAZIL                                    36
REUNION                                   20
NEW ZEALAND                               12
MEXICO                                     6
INDONESIA                                  6
COSTA RICA                                 4
ECUADOR                                    4
JAPAN                                      3
FRENCH POLYNESIA                           2
BAHAMAS                                    2
NEW CALEDONIA                              2
EL SALVADOR                                2
FIJI                                       2
VENEZUELA                                  2
SCOTLAND                                   1
OKINAWA                                    1
DOMINICAN REPUBLIC                         1
ST HELENA, British overseas territory      1
CANADA                                     1
ANGOLA    

In [170]:
df_sharks_country = df_sharks.groupby(['Country'])['Year'].count().sort_values(ascending=False).head()

In [171]:
df_sharks_country

Country
USA             563
AUSTRALIA       195
SOUTH AFRICA     95
BRAZIL           36
REUNION          20
Name: Year, dtype: int64

In [172]:
df_sharks_country.groupby(['Country']).get_group('USA')

Country
USA    563
Name: Year, dtype: int64

In [173]:
df_sharks_country.groupby(['Country']).get_group('AUSTRALIA')

Country
AUSTRALIA    195
Name: Year, dtype: int64

## Hipótesis 3
El número de casos ataques a surfistas aumenta cada año

In [174]:
df_sharks.Year.value_counts()

2015    42
2012    42
2016    40
2007    38
2008    38
        ..
1955     1
1957     1
1960     1
1963     1
0        1
Name: Year, Length: 70, dtype: int64

In [175]:
df_sharks.sort_values('Year', ascending=False)

,Case Number,Year,Type,Country,Area,Activity,Injury,Fatal (Y/N)
2,2018.06.09,2018,Invalid,USA,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,N
28,2018.04.15.b,2018,Unprovoked,AUSTRALIA,Western Australia,Surfing,Laceration to right thigh,N
3,2018.06.08,2018,Unprovoked,AUSTRALIA,New South Wales,Surfing,Minor injury to lower leg,N
52,2018.01.05,2018,Unprovoked,AUSTRALIA,Western Australia,Surfing,Foot bitten,N
51,2018.01.12,2018,Unprovoked,BRAZIL,Fernando de Noronha,Surfing,Minor injury to left forearm,N
...,...,...,...,...,...,...,...,...
5235,1922.01.04,1922,Unprovoked,AUSTRALIA,New South Wales,Surfing,"FATAL, disappeared, then his shark-bitten rema...",Y
5280,1918.03.22,1918,Unprovoked,AUSTRALIA,New South Wales,Surfing,"Severe laceration to arm, necessitating surgic...",N
6097,1828.00.00,1828,Unprovoked,USA,Hawaii,Surfing,FATAL,Y
6143,1779.00.00,1779,Unprovoked,USA,Hawaii,Surfing,"FATAL, buttock lacerated",Y


In [157]:
df_sharks.Injury.value_counts()

FATAL                                                          34
Foot bitten                                                    33
Left foot bitten                                               25
No injury, board bitten                                        22
Right foot bitten                                              16
                                                               ..
Lacerations and puncture wounds to hip, thigh, arm and hand     1
Toothmarks in board & his swim trunks                           1
3-inch laceration to right foot                                 1
Injuries to left thigh, calf & foot                             1
Minor injury                                                    1
Name: Injury, Length: 615, dtype: int64

In [ ]:
#df_sharks.Injury = df_sharks.Injury.str.replace(r LO QUE SEA DISTINTO DE FATAL ---> NO FATAL???

In [163]:
df_sharks["Fatal (Y/N)"].value_counts()

N          876
Y           49
UNKNOWN      6
Name: Fatal (Y/N), dtype: int64